In [ ]:
pip install jugaad-data pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.13.5
    Uninstalling beautifulsoup4-4.13.5:
      Successfully uninstalled beautifulsoup4-4.13.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 3.1.2 requires click>=8.1.3, but you have click 7.1.2 which is incompatible.
dask 2025.12.0 requires click>=8.1, but you have click 7.1.2 which is incompatible.
typer-slim 0.20.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.
typer 0.20.0 requires click>=8.0.0, but you have click 7.1.2 which is incomp

In [ ]:
pip install nse-workday

In [ ]:
from datetime import date, timedelta
import os
import pandas as pd
from jugaad_data.nse import full_bhavcopy_save
from nse_workday import isHoliday
import time
import random

# define dynamic date range: from exactly one year ago to today
today = date.today()
start_date = today - timedelta(days=365)

# define folder path for saving files and master file name
save_folder = "/content/dbhavcopies"
master_file = "/content/master_bhav.csv"  # Master CSV file path

# create folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

# initialize master DataFrame, loading existing data if available
if os.path.exists(master_file) and os.path.getsize(master_file) > 0:
    master_df = pd.read_csv(master_file)
else:
    print("Master CSV is empty or does not exist. Creating a new one.")
    master_df = pd.DataFrame()

# loop through each date in the dynamic 1-year range
current_date = start_date
while current_date <= today:
    # skip weekends (Saturday & Sunday)
    if current_date.weekday() in (5, 6):
        print(f"Skipping {current_date} (Weekend)")
        current_date += timedelta(days=1)
        continue

    # check if the current date is a trading holiday using isHoliday
    if isHoliday(current_date.strftime("%d-%m-%Y")):
        print(f"Skipping {current_date} (Market Holiday)")
        current_date += timedelta(days=1)
        continue

    try:
        print(f"Downloading bhavcopy for {current_date}...")

        # download bhavcopy and save in the specified folder
        full_bhavcopy_save(current_date, save_folder)

        # expected file name pattern is "cm01JAN2025bhav.csv"
        file_name = f"sec_bhavdata_full_{current_date.strftime('%d%b%Y')}bhav.csv"
        file_path = os.path.join(save_folder, file_name)

        # print expected file path - helps with debugging as well
        print(f"Looking for file: {file_path}")

        # check if file exists and is not empty, then append its data
        if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
            df = pd.read_csv(file_path)

            master_df = pd.concat([master_df, df], ignore_index=True)
            print(f"Appended data for {current_date} to master file.")
        else:
            print(f"File for {current_date} not found or is empty.")

    except Exception as e:
        print(f"Error: Bhavcopy for {current_date} is not available. Error: {e}")

    # move to the next day
    current_date += timedelta(days=1)


# save the master DataFrame to CSV
if not master_df.empty:
    master_df.to_csv(master_file, index=False)
    print(f"Master CSV saved at {master_file} with {len(master_df)} records.")
else:
    print("No data found! Master CSV not created.")


Master CSV is empty or does not exist. Creating a new one.
Looking for file: /content/dbhavcopies/sec_bhavdata_full_26Dec2024bhav.csv
✅ Appended data for 2024-12-26 to master file.
Looking for file: /content/dbhavcopies/sec_bhavdata_full_27Dec2024bhav.csv
✅ Appended data for 2024-12-27 to master file.
Skipping 2024-12-28 (Weekend)
Skipping 2024-12-29 (Weekend)
Looking for file: /content/dbhavcopies/sec_bhavdata_full_30Dec2024bhav.csv
✅ Appended data for 2024-12-30 to master file.
Looking for file: /content/dbhavcopies/sec_bhavdata_full_31Dec2024bhav.csv
✅ Appended data for 2024-12-31 to master file.
Looking for file: /content/dbhavcopies/sec_bhavdata_full_01Jan2025bhav.csv
✅ Appended data for 2025-01-01 to master file.
Looking for file: /content/dbhavcopies/sec_bhavdata_full_02Jan2025bhav.csv
✅ Appended data for 2025-01-02 to master file.
Looking for file: /content/dbhavcopies/sec_bhavdata_full_03Jan2025bhav.csv
✅ Appended data for 2025-01-03 to master file.
Skipping 2025-01-04 (Weeken